## Drowsiness Detection OpenCV
This code can detect your eyes and alert when the user is drowsy.
## Applications
This can be used by riders who tend to drive for a longer period of time that may lead to accidents.
## Methodology
This uses various python libraries such as scipy, dlib,opencv and a package of opencv called imutils. This also uses a face and eye cascade which detects the face and eyes. The drowsiness of a driver is detected by the standard formula of the diameter of how wide the eye is open.
### Made by Jamal Ahmed Khan, SE department, NED University


## How to run the program
To run this program all the dependancies must be installed prior to the execution of code and the cascade file should be present in the directory of the .ipynb file.

In [6]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [7]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [8]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [9]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [11]:
cap=cv2.VideoCapture(0)
flag=0
while True:
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if ear < thresh:
            flag += 1            
            if flag >= frame_check:
                cv2.putText(frame, "*ALERT!Driver is Drowsy Stop The Car*", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)                
                cv2.putText(frame, "****************ALERT!****************", (10,325),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):#If q button is pressed on keyboard the program will exit
        cv2.destroyAllWindows()
        cap.release()
        break